In [59]:
from Bio import SeqIO
import pandas as pd
import json

In [60]:
input_fasta = 'covid_sequence.fa'

In [61]:
orfs={
    '1A':{'START_NUMBER':266,'END_NUMBER':13483,'START_CODON':'ATG','END_CODON':'TAA'},
    '1AB':{'START_NUMBER':266,'END_NUMBER':21555,'START_CODON':'ATG','END_CODON':'TAA'},
    'SPIKE':{'START_NUMBER':21563,'END_NUMBER':25384,'START_CODON':'ATG','END_CODON':'TAA'},
    '3A':{'START_NUMBER':25393,'END_NUMBER':26220,'START_CODON':'ATG','END_CODON':'TAA'},
    '4':{'START_NUMBER':26245,'END_NUMBER':26472,'START_CODON':'ATG','END_CODON':'TAA'},
    '5':{'START_NUMBER':26523,'END_NUMBER':27191,'START_CODON':'ATG','END_CODON':'TAA'},
    '6':{'START_NUMBER':27202,'END_NUMBER':27387,'START_CODON':'ATG','END_CODON':'TAA'},
    '7A':{'START_NUMBER':27394,'END_NUMBER':27759,'START_CODON':'ATG','END_CODON':'TGA'},
    '7B':{'START_NUMBER':27756,'END_NUMBER':27887,'START_CODON':'ATG','END_CODON':'TAA'},
    '8':{'START_NUMBER':27894,'END_NUMBER':28262,'START_CODON':'ATG','END_CODON':'TAA'},
    '9':{'START_NUMBER':28281,'END_NUMBER':29540,'START_CODON':'ATG','END_CODON':'TAA'},
    '10':{'START_NUMBER':29565,'END_NUMBER':29681,'START_CODON':'ATG','END_CODON':'TAG'}
}

In [36]:
orfs={
    '1A':{'START_NUMBER':266,'END_NUMBER':13483,'START_CODON':'ATG','END_CODON':'TAA'},
    '1AB':{'START_NUMBER':266,'END_NUMBER':21555,'START_CODON':'ATG','END_CODON':'TAA'}
}

In [62]:
def write_fasta(output_file,name,sequence):
    text = f'>{name}\n{sequence}\n'
    file =  open(output_file,'a')
    file.write(text)
    file.close()

In [63]:
quality_df = pd.DataFrame(columns=['SAMPLE','1A','1AB','SPIKE', '3A', '4', '5', '6', '7A', '7B', '8', '9', '10'])
count_df = {'1A':0,'1AB':0,'SPIKE':0,'3A':0,'4':0,'5':0,'6':0,'7A':0,'7B':0,'8':0,'9':0,'10':0}

In [64]:
fasta_sequences = SeqIO.parse(open(input_fasta),'fasta')
total = 0

for fasta in fasta_sequences:
    total += 1
    #PEGANDO NOME E SEQUENCIA DAS AMOSTRAS 
    name, sequence = fasta.id, str(fasta.seq)
    quality_df = quality_df.append({'SAMPLE':name},ignore_index=True)
    
    for orf in orfs.keys():
        #DECLARAÇÃO DE VARIAVEIS UNICAS PRA CADA ORF
        #Arquivos
        output_file_quality = f'./output/quality_ORF_{orf}.csv'
        output_file_fasta = f'./output/ORF_{orf}.fasta'

        #posições genomicas
        start_codon = orfs[orf]['START_CODON']
        start_posix = orfs[orf]['START_NUMBER'] - 1
        end_codon = orfs[orf]['END_CODON']
        end_posix = orfs[orf]['END_NUMBER']
        
        #ORF
        orf_seq = sequence[start_posix : end_posix]
        if orf == '1AB':
            orf_seq = orf_seq[:13202]+"C"+orf_seq[13202:]
            
        #ANALISE DE PARAMETROS DE QUALIDADE
        gaps = orf_seq.count('-')
        nbase = orf_seq.count('N')
        lenght = len(orf_seq)

        #TESTA SE O START CODON E O END CODON SÃO OS ESPERADOS
        if not (orf_seq[:3] == start_codon and orf_seq[-3:] == end_codon):
            print(f'Sample:{name}\tORF:{orf}\tstart:{orf_seq[:10]}\tend:{orf_seq[-10:]}')
            write_fasta(f'problems_{orf}.fa',name,sequence)

            
        if not "N" in orf_seq:
            #SALVANDO AS SEQUENCIAS
            write_fasta(output_file_fasta,name,orf_seq)
            quality_df.loc[quality_df['SAMPLE'] == name, orf] = f'ACEPTED'

        else:
            quality_df.loc[quality_df['SAMPLE'] == name, orf] = f'REJECTED'
            count_df[orf] += 1

Sample:hCoV-19/Brazil/AM-FIOCRUZ-20143088ER/2020	ORF:10	start:ATGGGCTATA	end:----------
Sample:hCoV-19/Brazil/SP-IB_100744/2021	ORF:7A	start:ATGAAAATTA	end:GACAGAATTA
Sample:hCoV-19/Brazil/SP-IB_100744/2021	ORF:7B	start:ATTATTGAAC	end:TCACGCCTAA
Sample:hCoV-19/Brazil/SP-IB_100745/2021	ORF:10	start:ATGGGCTATA	end:TCTCACATA-
Sample:hCoV-19/Brazil/SP-IB_100746/2021	ORF:7A	start:ATGAAAATTA	end:NNNNNNNNNN
Sample:hCoV-19/Brazil/SP-IB_100746/2021	ORF:7B	start:NNNNNNNNNN	end:TCACGCCTAA
Sample:hCoV-19/Brazil/SP-IB_100840/2021	ORF:7A	start:ATGAAAATTA	end:GACAGAATGT
Sample:hCoV-19/Brazil/SP-IB_100842/2021	ORF:7A	start:ATGAAAATTA	end:GACAGAATGT
Sample:hCoV-19/Brazil/SP-IB_101223/2021	ORF:7A	start:ATGAAAATTA	end:NNNNNNNNNN
Sample:hCoV-19/Brazil/SP-IB_101223/2021	ORF:7B	start:NNNNNNNNNN	end:TCACGCCTAA
Sample:hCoV-19/Brazil/SP-IB_101283/2021	ORF:7A	start:ATGAAAATTA	end:NNNNNNNNNN
Sample:hCoV-19/Brazil/SP-IB_101283/2021	ORF:7B	start:NNNNNNNNNN	end:TCACGCCTAA
Sample:hCoV-19/Brazil/SP-IB_101284/2021	ORF

In [65]:
quality_df.loc[quality_df['1AB'] == 'REJECTED']

,SAMPLE,1A,1AB,SPIKE,3A,4,5,6,7A,7B,8,9,10
8,hCoV-19/Brazil/BA-50/2021,REJECTED,REJECTED,REJECTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED
13,hCoV-19/Brazil/BA-56/2021,REJECTED,REJECTED,REJECTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,REJECTED,ACEPTED
15,hCoV-19/Brazil/BA-49/2021,REJECTED,REJECTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED
134,hCoV-19/Brazil/MG-LBI229/2021,REJECTED,REJECTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED
143,hCoV-19/Brazil/MG-LBI263/2021,REJECTED,REJECTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4289,hCoV-19/Brazil/SP-IB_103575/2021,REJECTED,REJECTED,REJECTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED
4292,hCoV-19/Brazil/SP-IB_101596/2021,REJECTED,REJECTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED
4293,hCoV-19/Brazil/SP-IB_101597/2021,REJECTED,REJECTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED
4294,hCoV-19/Brazil/SP-IB_101598/2021,REJECTED,REJECTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED,ACEPTED


In [67]:
quality_df.to_csv('ORF_quality.csv',header=True,sep='\t',index=False)

In [70]:
for orf in count_df.keys():
    print(f'ORF:{orf} - Rejected samples: {count_df[orf]}')

ORF:1A - Rejected samples: 776
ORF:1AB - Rejected samples: 859
ORF:SPIKE - Rejected samples: 115
ORF:3A - Rejected samples: 3
ORF:4 - Rejected samples: 0
ORF:5 - Rejected samples: 0
ORF:6 - Rejected samples: 1
ORF:7A - Rejected samples: 37
ORF:7B - Rejected samples: 32
ORF:8 - Rejected samples: 17
ORF:9 - Rejected samples: 53
ORF:10 - Rejected samples: 8
